## crawling page

### 라이브러리 설치

```
pip install bs4
pip install pymysql
pip install selenium
```

### chromewebdriver 설치

https://sites.google.com/a/chromium.org/chromedriver/downloads

--- 

참고 : https://greeksharifa.github.io/references/2020/10/30/python-selenium-usage/

In [4]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
import re
import os
import urllib
import numpy as np
import pandas as pd
import pymysql

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
if canfetch("https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List") == False:
    print('[Error] ' + url)
else:
    print('GoGo')

GoGo


In [225]:
db_info = pymysql.connect(user='root', passwd='root',host='localhost',port=3306, db='test')
cursor = db_info.cursor()

In [99]:
sql = "SELECT * FROM category;"
cursor.execute(sql)
result = pd.DataFrame(cursor.fetchall())
result.columns = ['id','name']
result

,id,name
0,1,소설/시/희곡
1,74,역사
2,170,경제경영
3,336,자기계발
4,351,컴퓨터/모바일
5,517,예술/대중문화
6,656,인문학
7,798,사회과학
8,987,과학
9,1108,어린이


In [213]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime

driver = webdriver.Chrome('/SSAFY/python_project/chromedriver')
driver.get('https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail&PublishMonth=36&SortOrder=2&Stockstatus=0&CID=1&page=1')

In [92]:
ReviewList = driver.find_elements_by_css_selector('#CommentReviewList .hundred_list')

for rl in ReviewList:
    star = rl.find_elements_by_css_selector('img')
    rank = 0
    for st in star:
        st_split = st.get_attribute('src').split('/')[-1].split('.')[0]
        if st_split=='icon_star_on':
            rank+=2
    review = rl.find_element_by_css_selector('li > div > div > a').text
    nickname = rl.find_element_by_css_selector('div[class=left] > a').text
    reg_time = rl.find_element_by_css_selector('div[class=left] > span').text
    print(nickname)
    print(datetime.strptime(reg_time,'%Y-%m-%d'))
    print(review)
    print(rank)
    break;

잠자냥
2021-07-22 00:00:00
미드나잇 도서관이 있다면 이 책을 읽기 전으로 돌려주세요, 돈도 시간도 무쟈게 아깝습니다! 후회합니다! 도덕교과서도 아니고 읽는 내내 지겨워 죽는 줄 알았다. 이렇게 엉망인 소설도 진짜 오랜만이다. 전세계 독자들은 대체 이 책의 무엇에 반한 걸까? 베스트셀러에 대한 불신만 한번 더 깊어졌다.
2


In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait


url = 'https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail&PublishMonth=36&SortOrder=2&Stockstatus=0&CID='
detail_url = 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId='

book_sql = "INSERT INTO book (isbn, title, coverFilepath, category_id) VALUES (%s, %s, %s, %s)"
review_sql = "INSERT INTO review (rank, content, reg_time, user_id, book_id) VALUES (%s, %s, %s, %s, %s)"
user_log_sql = "INSERT INTO user_log (reg_time, user_id, book_id) VALUES (%s, %s, %s)"
user_same_sql = "SELECT * FROM user where nickname = %s;"
user_insert_sql = "INSERT INTO user (id, password, nickname) VALUES (%s, %s, %s)"
book_id = 1
user_c = 1
for r in result['id']:

    for i in range(1,21):
        driver.get(url + str(r) + '&page=' + str(i))
        time.sleep(1)
        ss_book_boxs = driver.find_elements_by_class_name('ss_book_box')

        for ss in ss_book_boxs:
            time.sleep(0.5)
            isbn = ss.find_element_by_class_name('Search3_Result_SafeBasketArea').get_attribute('isbn')
            title = ss.find_element_by_css_selector('a[class=bo3] > b').text
            try:
                filepath = ss.find_element_by_class_name('i_cover').get_attribute('src')
            except NoSuchElementException as e:
                continue
            try:
                cursor.execute(book_sql,(isbn, title, filepath, r))
                book_id = cursor.lastrowid

                time.sleep(0.1)

                ss.find_element_by_css_selector('a[class=bo3]').send_keys(Keys.CONTROL + "\n")

                driver.switch_to.window(driver.window_handles[1])

                time.sleep(0.1)

                body = driver.find_element_by_tag_name('body')
                body.send_keys(Keys.END)

                all_review = wait(driver, 10).until(EC.presence_of_element_located((By.ID, 'tabTotal')))

                driver.execute_script("arguments[0].click();", all_review)
                time.sleep(0.1)
            except TimeoutException:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                continue
 
            try:
                
                total = 0
                while len(driver.find_element_by_css_selector('#divReviewPageMore').get_property('style'))==0 and total<10:

                    driver.find_element_by_css_selector('#divReviewPageMore > div > a').send_keys(Keys.ENTER)
                    total+=1
                    time.sleep(0.3)

            except (StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException):
                pass
            
            
            try:
                ReviewList = driver.find_elements_by_css_selector('#CommentReviewList .hundred_list')

                for rl in ReviewList:

                    star = rl.find_elements_by_css_selector('img')
                    rank = 0
                    for st in star:
                        st_split = st.get_attribute('src').split('/')[-1].split('.')[0]
                        if st_split=='icon_star_on':
                            rank+=2
                    review = rl.find_element_by_css_selector('li > div > div > a').text
                    nickname = rl.find_element_by_css_selector('div[class=left] > a').text
                    reg_time = rl.find_element_by_css_selector('div[class=left] > span').text

                    if len(review)>300: continue
                    cursor.execute(user_same_sql,nickname)
                    user = cursor.fetchone()

                    if cursor.fetchone() == None:
                        cursor.execute(user_insert_sql,('user' + str(user_c), str(user_c).zfill(6), nickname))
                        user_id = cursor.lastrowid
                        user_c+=1
                    else:
                        user_id = user[0]

                    cursor.execute(review_sql,(str(rank), review, reg_time, str(user_id), str(book_id)))
                    cursor.execute(user_log_sql,(reg_time, str(user_id), str(book_id)))
            except (StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException):
                pass
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

db_info.commit()

In [229]:
driver.close()

In [ ]:
db_info.commit()

In [224]:
db_info.close()